# 2nd Level Model Structure: Reactive Stage

In [1]:
import sys
import os
import dill
import pickle
sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../..'))
import numpy as np
from matplotlib import pyplot as plt

In [2]:
from pyomo import environ as pe
from global_sets.component import m
from utility.display_utility import trans_product_mole, trans_product_mass
from utility.model_utility import add_dual, update_dual, check_DOF, augmented_objective, add_solver,\
                                    select_MPCC, which_MPCC, disable_restoration
from utility.data_utility import cal_cnumber

# stage construction rules
from physics.kinetics.kinetics_reactive import kinetic_block_rule
from physics.energy.energy_reactive import energy_block_rule
from physics.VLE.VLE_reactive_MPCC_P import VLE_block_rule
from physics.MPCC.MPCC_P import P_NCP_block_rule, P_Reg_block_rule, P_pf_block_rule

In [3]:
model = pe.ConcreteModel()

# Global Sets (Inlet / Outlet)

In [4]:
model.inlet = pe.Set(initialize=['in','R'])
model.outlet = pe.Set(initialize=['out','P'])
model.stream = model.inlet | model.outlet

In [5]:
model.comp_naphtha = pe.Set(initialize = [i for i in m.COMP_ORG if cal_cnumber(i) >= 5 and cal_cnumber(i) <= 7])
model.comp_gasoline = pe.Set(initialize = [i for i in m.COMP_ORG if cal_cnumber(i) >= 8 and cal_cnumber(i) <= 12])
model.comp_diesel = pe.Set(initialize = [i for i in m.COMP_ORG if cal_cnumber(i) >= 13 and cal_cnumber(i) <= 18])
model.comp_heavy = pe.Set(initialize = [i for i in m.COMP_ORG if cal_cnumber(i) >= 19 and cal_cnumber(i) <= 56])

# Global Variables

In [6]:
# Tray Inlet/Outlet Variable
model.x_ = pe.Var(model.inlet,m.COMP_TOTAL,within=pe.NonNegativeReals)
model.y_ = pe.Var(model.inlet,m.COMP_TOTAL,within=pe.NonNegativeReals)
model.x = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals)
model.y = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals)
model.z = pe.Var(m.COMP_FEED,within=pe.NonNegativeReals)

model.L = pe.Var(model.stream,within=pe.NonNegativeReals)
model.V = pe.Var(model.stream,within=pe.NonNegativeReals)
model.F = pe.Var(within=pe.NonNegativeReals)

model.H_L_ = pe.Var(model.inlet,within=pe.Reals)
model.H_V_ = pe.Var(model.inlet,within=pe.Reals)
model.H_L = pe.Var(within=pe.Reals)
model.H_V = pe.Var(within=pe.Reals)
model.H_F = pe.Var(within=pe.Reals)

# State Variable
model.T = pe.Var(within=pe.NonNegativeReals,bounds=(200+273.15,300+273.15)) # K
model.T_F = pe.Var(within=pe.NonNegativeReals) # K
model.P = pe.Var(within=pe.NonNegativeReals,bounds=(10,30)) # Bar

model.f_V = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals,initialize=1e-20)
model.f_L = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals,initialize=1e-20)

model.cat = pe.Var(within=pe.NonNegativeReals,initialize=30000) # kg
model.Q_main = pe.Var(within=pe.Reals) # MW
model.r_total_comp = pe.Var(m.COMP_TOTAL,within=pe.Reals) # kmol/s

In [7]:
model.P_naphtha = pe.Var(within=pe.NonNegativeReals)
model.P_gasoline = pe.Var(within=pe.NonNegativeReals)
model.P_diesel = pe.Var(within=pe.NonNegativeReals)
model.P_heavy = pe.Var(within=pe.NonNegativeReals)

# Construct Individual Blocks

In [8]:
if model.find_component('kinetics_block'):
    print('haha')

In [9]:
model.kinetics_block = pe.Block(rule=kinetic_block_rule)

> Importing Kinetics Blocks......
> Adding the following local variable:
--------------------------------------------------
| kinetics_block.k_FT
| kinetics_block.r_FT_total
| kinetics_block.g0_FT
| kinetics_block.alpha
| kinetics_block.r_FT_cnum
| kinetics_block.r_FT_comp
| kinetics_block.k_WGS
| kinetics_block.Ke_WGS
| kinetics_block.r_WGS
| kinetics_block.r_WGS_comp
--------------------------------------------------



In [10]:
model.energy_block = pe.Block(rule=energy_block_rule)

> Importing Energy Blocks......
> Adding the following local variable:
--------------------------------------------------
| energy_block.dH_F
| energy_block.dH_V
| energy_block.dH_L
| energy_block.dH_vap
--------------------------------------------------



In [11]:
model.VLE_block = pe.Block(rule=VLE_block_rule)

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| VLE_block.P_VLE
| VLE_block.n_ave
| VLE_block.n_ave_cal
| VLE_block.Hen
| VLE_block.Hen0
| VLE_block.gamma
| VLE_block.P_sat
| VLE_block.P_sat_Y
| VLE_block.P_sat_dY_inf
| VLE_block.P_sat_dY0
| VLE_block.Hen_ref
| VLE_block.Hen0_ref
| VLE_block.gamma_ref
| VLE_block.V_L
| VLE_block.V_L_dY_inf
| VLE_block.V_L_dY0
| VLE_block.poynting
--------------------------------------------------



# Standard MESH Equations

## Mass Balance

\begin{equation}
Fz_{i}+\sum_{s\in{inlet}}L_{s}x'_{i,s}+\sum_{s\in{inlet}}V_{s}y'_{i,s}-\sum_{s\in{outlet}}L_{s}x_{i,s}+\sum_{s\in{ouelet}}V_{s}y_{i,s}+R_{i} = 0 \\
i = 1,...NC
\end{equation}

In [12]:
def mass_balance_main_rule(model,i):
    if i in m.COMP_FEED:
        return model.F*model.z[i] + sum(model.L[s]*model.x_[s,i] + model.V[s]*model.y_[s,i] for s in model.inlet)\
        + model.r_total_comp[i] - sum(model.L[s]*model.x[i] + model.V[s]*model.y[i] for s in model.outlet) == 0
    else:
        return sum(model.L[s]*model.x_[s,i] + model.V[s]*model.y_[s,i] for s in model.inlet)\
        + model.r_total_comp[i] - sum(model.L[s]*model.x[i] + model.V[s]*model.y[i] for s in model.outlet) == 0
model.mass_balance_main_con = pe.Constraint(m.COMP_TOTAL,rule=mass_balance_main_rule)

## Equilibrium

\begin{align}
&f_{i,V} = f_{i,L} \\
&i = 1,...NC
\end{align}

In [13]:
def VL_equil_rule(model,i):
    return model.f_V[i] == model.f_L[i]
model.VL_equil_con = pe.Constraint(m.COMP_TOTAL,rule=VL_equil_rule)

## Summation

\begin{equation}
\sum_ix_{i} = \sum_iy_{i} \\
F + L_{in} + V_{in} + R = L_{out} + V_{out}
\end{equation}

In [14]:
def summation_x_y_rule(model):
    return sum(model.x[i] for i in m.COMP_TOTAL) == sum(model.y[i] for i in m.COMP_TOTAL)
model.summation_x_y_con = pe.Constraint(rule=summation_x_y_rule)

def summation_total_mass_rule(model):
    return model.F + sum(model.L[s] + model.V[s] for s in model.inlet) + sum(model.r_total_comp[i] for i in m.COMP_TOTAL)\
            - sum(model.L[s] + model.V[s] for s in model.outlet) == 0
model.summation_total_mass_con = pe.Constraint(rule=summation_total_mass_rule)

## Energy Balance

\begin{equation}
F H_f+\sum_{s\in{inlet}}L_{s}H_{l,s}+\sum_{s\in{inlet}}V_{s}H_{v,s}-\sum_{s\in{outlet}}L_{s}H_{l,s}-\sum_{s\in{outlet}}V_{s}H_{v,s}+Q = 0
\end{equation}

In [15]:
def heat_balance_main_rule(model):
    return model.F*model.H_F + sum(model.L[s]*model.H_L_[s] + model.V[s]*model.H_V_[s] for s in model.inlet) \
            + model.Q_main - sum(model.L[s]*model.H_L + model.V[s]*model.H_V for s in model.outlet) == 0
model.heat_balance_main_con = pe.Constraint(rule=heat_balance_main_rule)

# Product Amount / Total Revenue

In [16]:
def P_naphtha_rule(model):
    return model.P_naphtha == model.V['out'] * sum(model.y[i] for i in model.comp_naphtha) + \
                              model.L['out'] * sum(model.x[i] for i in model.comp_naphtha)
model.P_naphtha_con = pe.Constraint(rule=P_naphtha_rule)

def P_gasoline_rule(model):
    return model.P_gasoline == model.V['out'] * sum(model.y[i] for i in model.comp_gasoline) + \
                               model.L['out'] * sum(model.x[i] for i in model.comp_gasoline)
model.P_gasoline_con = pe.Constraint(rule=P_gasoline_rule)

def P_diesel_rule(model):
    return model.P_diesel == model.V['out'] * sum(model.y[i] for i in model.comp_diesel) + \
                             model.L['out'] * sum(model.x[i] for i in model.comp_diesel)
model.P_diesel_con = pe.Constraint(rule=P_diesel_rule)

def P_heavy_rule(model):
    return model.P_heavy == model.V['out'] * sum(model.y[i] for i in model.comp_heavy) + \
                             model.L['out'] * sum(model.x[i] for i in model.comp_heavy)
model.P_heavy_con = pe.Constraint(rule=P_heavy_rule)

## MPCC: Add all three

In [17]:
model.MPCC_P_pf = pe.Block(rule = P_pf_block_rule)
model.MPCC_P_NCP = pe.Block(rule = P_NCP_block_rule)
model.MPCC_P_Reg = pe.Block(rule = P_Reg_block_rule)

> Importing MPCC_P_pf Blocks......
> Adding the following local variable:
--------------------------------------------------
| MPCC_P_pf.s_L
| MPCC_P_pf.s_V
| MPCC_P_pf.pf
| MPCC_P_pf.epi
| MPCC_P_pf.rho
--------------------------------------------------
> Spliting pressure used in VLE
Deleted original P_equal constraint

> Deleted kinetics rates constraints
> Added f_V_MPCC, updated rates constraints

> Importing MPCC_P_NCP Blocks......
> Adding the following local variable:
--------------------------------------------------
| MPCC_P_NCP.s_L
| MPCC_P_NCP.s_V
| MPCC_P_NCP.epi
--------------------------------------------------
> Adding complementarity constraint, spliting pressure used in VLE
> No constraint to delete

> Already replaced f_V_MPCC

> Importing MPCC_P_Reg Blocks......
> Adding the following local variable:
--------------------------------------------------
| MPCC_P_Reg.s_L
| MPCC_P_Reg.s_V
| MPCC_P_Reg.epi
--------------------------------------------------
> Adding comple

In [18]:
select_MPCC(model,'pf')

> Selected MPCC: MPCC_P_pf
s_L:  0
s_V:  0



# Testing

In [19]:
add_dual(pe,model)

Created the follow pyomo suffixes:
ipopt_zL_out, ipopt_zU_out, ipopt_zL_in, ipopt_zU_in, dual


## Fixing In/Product Redundent Flow

In [20]:
model.x_.fix(0)
model.y_.fix(0)
model.L['in'].fix(0)
model.L['R'].fix(0)
model.L['P'].fix(0)

model.V['in'].fix(0)
model.V['R'].fix(0)
model.V['P'].fix(0)
model.H_L_.fix(0)
model.H_V_.fix(0)

## Fixing Model Parameters

In [21]:
model.cat.fix(30000)
model.P.fix(20)
model.T_F.fix(200+273.15)
model.F.fix(10)
model.z['CO'].fix(1/(1+2)-0/2)
model.z['H2'].fix(2/(1+2)-0/2)
model.z['C30H62'].fix(0)
model.VLE_block.n_ave.fix(20)

In [22]:
model.T.setub(300+273.15)

# Objective

In [23]:
# model.obj = augmented_objective(pe, model, expr = 43*model.P_naphtha + 90*model.P_gasoline + 128*model.P_diesel + \
#                                 100*model.P_heavy, sense=pe.maximize)
model.obj = augmented_objective(pe, model, expr = 54*model.P_naphtha + 112*model.P_gasoline + 161*model.P_diesel + \
                                350*model.P_heavy, sense=pe.maximize)

------------------------------------------------------------------------------------------------------------
> Obj = maximize
> 54*P_naphtha + 112*P_gasoline + 161*P_diesel + 350*P_heavy - MPCC_P_pf.pf
------------------------------------------------------------------------------------------------------------


In [24]:
opt = add_solver(pe, max_iter = 5000, warm_start = False, output = False)
disable_restoration(mode = 'enable')

In [25]:
results = opt.solve(model,tee=True)
update_dual(pe,model)

Ipopt 3.12.10: print_user_options=yes
tol=1e-08
option_file_name=./ipopt.opt
max_iter=5000

Using option file "./ipopt.opt".


List of user-set options:

                                    Name   Value                used
                           compl_inf_tol = 1                     yes
                         constr_viol_tol = 1                     yes
                            dual_inf_tol = 1000                  yes
                           linear_solver = mumps                 yes
                              ma86_small = 1e-30                  no
                                  ma86_u = 0.0001                 no
                              ma97_order = best                   no
                              ma97_small = 1e-30                  no
                            ma97_switch1 = at_start               no
                            ma97_switch2 = od_hd_reuse            no
                                  ma97_u = 1e-06                  no
                  

  74  1.3460892e+02 6.35e+02 1.04e+06  -1.0 3.78e+03    -  2.89e-03 7.88e-05h  1
  75  1.6323508e+02 6.31e+02 9.07e+05  -1.0 4.28e+03    -  2.73e-05 6.62e-03h  1
  76  1.6351792e+02 6.31e+02 9.07e+05  -1.0 3.90e+03    -  2.53e-03 7.17e-05h  1
  77  2.2218168e+02 6.24e+02 8.89e+05  -1.0 4.96e+03    -  1.86e-05 1.17e-02h  1
  78  2.2247649e+02 6.24e+02 8.89e+05  -1.0 4.13e+03    -  2.45e-03 7.08e-05h  1
  79  2.6666345e+02 6.19e+02 8.79e+05  -1.0 5.25e+03    -  1.90e-05 8.35e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  80  2.6693452e+02 6.19e+02 8.79e+05  -1.0 4.47e+03    -  1.99e-03 6.01e-05h  1
  81  3.2027401e+02 6.13e+02 8.67e+05  -1.0 5.89e+03    -  1.56e-05 8.99e-03h  1
  82  3.2050671e+02 6.13e+02 8.67e+05  -1.0 5.24e+03    -  1.35e-03 4.40e-05h  1
  83  3.6825143e+02 6.09e+02 8.59e+05  -1.0 7.36e+03    -  1.10e-05 6.46e-03h  1
  84  3.6839785e+02 6.09e+02 8.59e+05  -1.0 1.13e+04    -  3.47e-04 1.29e-05h  1
  85  4.2570223e+02 6.06e+02

 175 -2.7291609e+01 1.45e-06 1.85e-02  -3.9 6.32e-02    -  1.00e+00 1.00e+00h  1
 176 -2.7291609e+01 1.38e-10 1.40e-09  -3.9 2.09e-05    -  1.00e+00 1.00e+00h  1
 177 -2.7292532e+01 4.15e-03 6.23e+03  -6.0 1.60e+01    -  9.88e-01 7.79e-01f  1
 178 -2.7292438e+01 3.66e-04 1.79e+00  -6.0 3.68e+00    -  1.00e+00 1.00e+00h  1
 179 -2.7292428e+01 2.28e-09 1.91e-06  -6.0 1.07e-02    -  1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 180 -2.7292431e+01 2.98e-07 1.10e-03  -7.6 1.05e-01    -  1.00e+00 1.00e+00h  1
 181 -2.7292431e+01 1.38e-10 5.55e-11  -7.6 5.30e-06    -  1.00e+00 1.00e+00h  1
 182 -2.7292431e+01 2.80e-10 1.04e-06  -9.0 3.23e-03    -  1.00e+00 1.00e+00h  1
 183 -2.7292431e+01 1.38e-10 1.20e-11  -9.0 5.07e-05    -  1.00e+00 1.00e+00h  1

Number of Iterations....: 183

                                   (scaled)                 (unscaled)
Objective...............:  -7.7978374566527062e+00   -2.7292431098284474e+01
Dual infea

In [26]:
# print('Component\t\tLiquid: {:.4f}\t\t\tVapor: {:.4f}'.format(model.L['out'].value,model.V['out'].value))
# print('-'*108)
# for i in model.x:
#     print('{:10s}'.format(i),'\t\t{:8.4%}\t\t\t{:8.4%}'.format(model.x[i].value,model.y[i].value))

In [27]:
# 43*model.P_naphtha.value + 90*model.P_gasoline.value + 128*model.P_diesel.value + 100*model.P_heavy.value
54*model.P_naphtha.value + 112*model.P_gasoline.value + 161*model.P_diesel.value + 350*model.P_heavy.value

27.292431091466298

In [28]:
model.T.value, model.VLE_block.P_VLE.value, model.kinetics_block.r_FT_total.value

(491.2052273637285, 20.000000253235335, 2.627918500626668)

In [29]:
model.P_naphtha.value, model.P_gasoline.value, model.P_diesel.value, model.P_heavy.value

(0.057730452372284705,
 0.0594576412400823,
 0.03655524418985241,
 0.03322953294259276)

In [30]:
# Conversion
(model.F.value - model.V['out'].value * (model.y['H2'].value + model.y['CO'].value) - \
model.L['out'].value * (model.x['H2'].value + model.x['CO'].value))/model.F.value

0.8041541716747378

In [31]:
# model.solutions.store_to(results)
# with open('../../saved_solutions/reactive_flash_MPCC_P_200C.pickle','wb') as f:
#     pickle.dump(results,f)

In [32]:
# with open('test_model.pickle','wb') as f:
#     dill.dump(model,f)